In [26]:
# Importando bibliotecas
import streamlit as st
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as ctx
import locale
import io
from io import BytesIO
from PyPDF2 import PdfWriter, PdfReader


In [27]:
# Configurando a nomenclatura da aba no navegador
st.set_page_config(page_title="MAXSATT - Plataforma de Monitoramento", layout="wide")

# Definir o título da página (um texto em fonte grande no topo da página)
#st.markdown("<h1 style='text-align:center;'font-size:40px;'>Plataforma de Monitoramento de Formigas por Sensoriamento Remoto</h1>", unsafe_allow_html=True)

# Adicionando a logo do Maxsatt na aba lateral
st.sidebar.image("logos/logotipo_Maxsatt.png", use_container_width=False, width=150)

# --- 1. Importando bases de dados ---
pred_attack = pd.read_parquet("prediction/Filtered_pred_attack.parquet")
stands_all = gpd.read_file("prediction/Talhoes_Manulife_2.shp")


2025-01-31 00:38:13.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:13.706 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:13.769 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:13.770 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [28]:

# --- 2. Tratando a base pred_attack ---
pred_attack['COMPANY'] = pred_attack['COMPANY'].str.upper()
pred_attack['FARM'] = pred_attack['FARM'].str.upper()
pred_attack['STAND'] = pred_attack['STAND'].str.upper()
pred_attack['DATE'] = pd.to_datetime(pred_attack['DATE']).dt.date

# --- 3. Tratando a base stands_all ---
stands_all = stands_all.to_crs("EPSG:32722")
stands_all['COMPANY'] = stands_all['Companhia'].str.upper()
stands_all['FARM'] = stands_all['Fazenda'].str.replace(" ", "_")
stands_all['STAND'] = stands_all.apply(lambda row: f"{row['Fazenda']}_{row['CD_TALHAO']}", axis=1)
stands_all['geometry'] = stands_all['geometry'].apply(lambda geom: geom.buffer(0) if not geom.is_valid else geom)
stands_all['stand_total_area_ha'] = stands_all['geometry'].area / 10000

# --- 4. Configurando os filtros ---
empresa = pred_attack['COMPANY'].unique()[0]
st.sidebar.write(f"**Empresa:** {empresa}")

fazenda = st.sidebar.selectbox('Selecione a Fazenda', options=pred_attack[pred_attack['COMPANY'] == empresa]['FARM'].unique())
talhao = st.sidebar.selectbox('Selecione o Talhão', options=pred_attack[(pred_attack['FARM'] == fazenda)]['STAND'].unique())
sorted_dates = sorted(pred_attack['DATE'].unique(), reverse=True)
data = st.sidebar.selectbox('Selecione a data', options=sorted_dates, index=0)

# --- 5. Bases filtradas com granularidades diferentes ---
filtered_data_company_date = pred_attack[(pred_attack['COMPANY'] == empresa) & (pred_attack['DATE'] == data)]
filtered_data_farm = pred_attack[(pred_attack['COMPANY'] == empresa) & (pred_attack['FARM'] == fazenda) & (pred_attack['DATE'] == data)]
filtered_data = pred_attack[(pred_attack['COMPANY'] == empresa) & (pred_attack['FARM'] == fazenda) & (pred_attack['STAND'] == talhao) & (pred_attack['DATE'] == data)]
filtered_farm = pred_attack[pred_attack['FARM'] == fazenda]
filtered_company = pred_attack[pred_attack['COMPANY'] == empresa]


# Using `filtered_data_company_date` for stand calculations to match with the structure
filtered_pred_attack = filtered_data_company_date

# --- 6. Aplicando os filtros ao stands_all ---
stands_all_filtered = stands_all[stands_all['COMPANY'] == empresa]
stands_all_filtered = stands_all_filtered.to_crs("EPSG:32722")
stands_sel = stands_all[stands_all['STAND'] == talhao]
stands_sel_farm = stands_all[stands_all['FARM'] == fazenda]


# --- 7. Calculando porcentagem de desfolha ---
QT = pred_attack['canopycov'].quantile(0.10)
filtered_pred_attack['Status'] = filtered_pred_attack['canopycov'].apply(lambda x: 'Desfolha' if x < QT else 'Saudavel')

stand_status_summary = (
    filtered_pred_attack.groupby(['COMPANY', 'FARM', 'STAND', 'Status'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

stand_status_summary['total_pixels'] = stand_status_summary['Desfolha'] + stand_status_summary['Saudavel']
stand_status_summary['desfolha_percentage'] = (stand_status_summary['Desfolha'] / stand_status_summary['total_pixels']) * 100

# --- 8. Mesclando os dados ---
merged_df = stand_status_summary.merge(stands_all_filtered[['FARM', 'STAND', 'stand_total_area_ha']], on=['FARM', 'STAND'], how='left')
merged_df['desfolha_area_ha'] = (merged_df['desfolha_percentage'] / 100) * merged_df['stand_total_area_ha']
merged_df = merged_df.sort_values(by='desfolha_area_ha', ascending=False)

# Base auxiliar com uma linha para cada fazenda
unique_area_per_farm = merged_df.drop_duplicates(subset=['FARM'])
unique_area_per_farm.rename(columns={'stand_total_area_ha': 'farm_total_area_ha'}, inplace=True)


# Juntando pred_attack com stands_all (sem filtros) e tratando a planilha resultante
merged_df_all = filtered_company.merge(stands_all_filtered[['FARM', 'STAND', 'geometry']], on=['FARM', 'STAND'], how='left')
merged_df_all = gpd.GeoDataFrame(merged_df_all, geometry='geometry', crs="EPSG:32722")
merged_df_all['Status'] = ['Desfolha' if x < QT else 'Saudavel' for x in merged_df_all['canopycov']]
merged_df_all['area_ha'] = merged_df_all['geometry'].area / 10000

# Criando a base agrupada por fazenda e status e tratando-a
grouped_farm = (merged_df_all.dropna(subset=['Status'])
                .groupby(['DATE', 'Status', 'FARM'])
                .agg(count=('Status', 'size'))
                .reset_index()
                .merge(unique_area_per_farm[['FARM', 'farm_total_area_ha']], on='FARM', how='left'))
grouped_farm['farm_desfolha_area_ha'] = grouped_farm['count']/100
grouped_farm['total'] = grouped_farm.groupby(['DATE', 'FARM'])['count'].transform('sum')
grouped_farm['percentage'] = (grouped_farm['count'] / grouped_farm['total']) * 100
grouped_farm = grouped_farm[grouped_farm['Status'] == 'Desfolha'].sort_values(by='DATE')
grouped_farm['farm_total_area_ha'] = grouped_farm['farm_total_area_ha'].round(1)
grouped_farm['percentage'] = grouped_farm['percentage'].round(1)

# Base filtrada para data selecionada
grouped_farm_date = grouped_farm[grouped_farm['DATE']==data]

# Base auxiliar com uma linha para cada talhão
unique_area_per_stand = merged_df.drop_duplicates(subset=['FARM', 'STAND'])
unique_area_per_stand.rename(columns={'area_ha': 'stand_total_area_ha'}, inplace=True)

# Criando a base agrupada por talhão e status e tratando-a
grouped_stand = (merged_df_all.dropna(subset=['Status'])
                .groupby(['DATE', 'Status', 'FARM', 'STAND'])
                .agg(count=('Status', 'size'))
                .reset_index()
                .merge(unique_area_per_stand[['STAND', 'stand_total_area_ha']], on='STAND', how='left'))
                
grouped_stand['stand_desfolha_area_ha'] = grouped_stand['count']/100
grouped_stand['total'] = grouped_stand.groupby(['DATE', 'FARM', 'STAND'])['count'].transform('sum')
grouped_stand['percentage'] = (grouped_stand['count'] / grouped_stand['total']) * 100
grouped_stand = grouped_stand[grouped_stand['Status'] == 'Desfolha'].sort_values(by='DATE')
grouped_stand['stand_total_area_ha'] = grouped_stand['stand_total_area_ha'].round(1)
grouped_stand['percentage'] = grouped_stand['percentage'].round(1)
grouped_stand = grouped_stand.drop_duplicates(subset=['DATE', 'FARM', 'STAND'])
grouped_stand = grouped_stand.sort_values(by='stand_desfolha_area_ha', ascending=False)

# Base filtrada para data selecionada
grouped_stand_date = grouped_stand[grouped_stand['DATE']==data]

# Base filtrada para a fazenda selecionada
grouped_stand_farm = grouped_stand[grouped_stand['FARM']==fazenda]
grouped_stand_farm = grouped_stand_farm.sort_values(by='stand_desfolha_area_ha', ascending=False)

# PREPARAR AS PLANILHAS PARA DOWNLOAD

# Planilha fazendas
# Criar coluna de mês e filtrar por data
grouped_farm['DATE'] = pd.to_datetime(grouped_farm['DATE'])
grouped_farm['Mes'] = grouped_farm['DATE'].dt.strftime('%b').str.lower()

# Criar coluna de porcentagem média de desfolha por mês, usando uma base auxiliar
average_farm = (
    grouped_farm
    .groupby(['FARM', 'Mes'])['percentage']
    .mean()
    .reset_index()
    .rename(columns={'percentage': 'Average%'}))
average_farm['Average%'] = average_farm['Average%'].round(1)
grouped_farm = grouped_farm.merge(average_farm, on=['FARM', 'Mes'], how='left')

# Criando as colunas de recomendação
grouped_farm['Si Monitorar'] = grouped_farm.apply(lambda row: row['farm_total_area_ha'] if row['Average%'] < 0.5 else None, axis=1)
grouped_farm['Controle 9M'] = grouped_farm.apply(lambda row: row['farm_total_area_ha'] if 0.5 <= row['Average%'] <= 5 else None, axis=1)
grouped_farm['Controle 3M'] = grouped_farm.apply(lambda row: row['farm_total_area_ha'] if row['Average%'] > 5 else None, axis=1)
grouped_farm = grouped_farm.sort_values(by=['FARM', 'DATE'])
grouped_farm['percentage_diff'] = grouped_farm.groupby('FARM')['percentage'].diff()
grouped_farm['percentage_diff'] = grouped_farm['percentage_diff'].round(1)
grouped_farm['Outra desfolha'] = grouped_farm.apply(lambda row: row['farm_total_area_ha'] if row['percentage_diff'] > 8 else None, axis=1)
grouped_farm.loc[grouped_farm['Outra desfolha'].notna(), 'Controle 3M'] = None

grouped_farm_temp = grouped_farm.copy()

grouped_farm_excel = grouped_farm.copy()
grouped_farm_excel = (grouped_farm_excel.rename(columns={
    'DATE': 'Data', 'FARM': 'Fazenda', 'farm_total_area_ha': 'Area total da fazenda', 
    'farm_desfolha_area_ha': 'Area total em desfolha', 'percentage': 'Porcentagem'}))

grouped_farm_excel.drop(columns = ['count', 'total', 'Mes', 'percentage_diff', 'Outra desfolha'], inplace=True)

# Definindo a planilha a ser exportada
#excel_farm = grouped_farm_excel.to_excel(index=False).encode('utf-8')

# Planilha talhões
# Criar coluna de mês
grouped_stand['DATE'] = pd.to_datetime(grouped_stand['DATE'])
grouped_stand['Mes'] = grouped_stand['DATE'].dt.strftime('%b').str.lower()

# Criar coluna de porcentagem média de desfolha por mês, usando uma base auxiliar
average_stand = (
    grouped_stand
    .groupby(['STAND', 'Mes'])['percentage']
    .mean()
    .reset_index()
    .rename(columns={'percentage': 'Average%'}))
average_stand['Average%'] = average_stand['Average%'].round(1)
grouped_stand = grouped_stand.merge(average_stand, on=['STAND', 'Mes'], how='left')

# Criando as colunas de recomendação
grouped_stand['Si Monitorar'] = grouped_stand.apply(lambda row: row['stand_total_area_ha'] if row['Average%'] < 0.5 else None, axis=1)
grouped_stand['Controle 9M'] = grouped_stand.apply(lambda row: row['stand_total_area_ha'] if 0.5 <= row['Average%'] <= 5 else None, axis=1)
grouped_stand['Controle 3M'] = grouped_stand.apply(lambda row: row['stand_total_area_ha'] if row['Average%'] > 5 else None, axis=1)
grouped_stand = grouped_stand.sort_values(by=['STAND', 'DATE'])
grouped_stand['percentage_diff'] = grouped_stand.groupby('STAND')['percentage'].diff()
grouped_stand['percentage_diff'] = grouped_stand['percentage_diff'].round(1)
grouped_stand['Outra desfolha'] = grouped_stand.apply(lambda row: row['stand_total_area_ha'] if row['percentage_diff'] > 8 else None, axis=1)
grouped_stand.loc[grouped_stand['Outra desfolha'].notna(), 'Controle 3M'] = None

grouped_stand_temp = grouped_stand.copy()

grouped_stand_excel = grouped_stand.copy()
grouped_stand_excel = (grouped_stand_excel.rename(columns={
    'DATE': 'Data', 'FARM': 'Fazenda', 'STAND': 'Talhao', 'stand_total_area_ha': 'Area total do talhao', 
    'stand_desfolha_area_ha': 'Area total em desfolha', 'percentage': 'Porcentagem'}))

grouped_stand_excel.drop(columns = ['count', 'total', 'Mes', 'percentage_diff', 'Outra desfolha'], inplace=True)

# Definindo a planilha a ser exportada
#excel_stand = grouped_stand_excel.to_excel(index=False).encode('utf-8')

2025-01-31 00:38:18.469 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:18.470 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:18.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:18.471 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:19.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:19.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:19.059 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-31 00:38:19.061 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [29]:
# TABELA RECOMENDAÇÃO GERAL

grouped_stand_data = grouped_stand[grouped_stand['DATE'].dt.date==data]

recommendations = {
    'Si Monitorar': 'Sem infestação: seguir monitorando',
    'Controle 9M': 'Infestação com baixa atividade: Controle nos próximos 9 meses',
    'Controle 3M': 'Infestação com média a alta atividade: Controle nos próximos 3 meses',
    'Outra desfolha': 'Aumento expressivo da desfolha: monitorar nos próximos meses'
}

recomendacao_geral = pd.DataFrame({
    'Recomendação': ['Si Monitorar', 'Controle 9M', 'Controle 3M', 'Outra desfolha'],
    'Área': [
        grouped_stand_data['Si Monitorar'].sum(),
        grouped_stand_data['Controle 9M'].sum(),
        grouped_stand_data['Controle 3M'].sum(),
        grouped_stand_data['Outra desfolha'].sum()
    ],
    'O que?': [recommendations['Si Monitorar'], recommendations['Controle 9M'], recommendations['Controle 3M'], recommendations['Outra desfolha']]
})

# TABELA RECOMENDAÇÃO FAZENDA

grouped_stand_tabela = grouped_stand_data[grouped_stand_data['FARM']==fazenda]

recomendacao_farm = pd.DataFrame({
    'Recomendação': ['Si Monitorar', 'Controle 9M', 'Controle 3M', 'Outra desfolha'],
    'Área': [
        grouped_stand_tabela['Si Monitorar'].sum(),
        grouped_stand_tabela['Controle 9M'].sum(),
        grouped_stand_tabela['Controle 3M'].sum(),
        grouped_stand_tabela['Outra desfolha'].sum()
    ],
    'O que?': [recommendations['Si Monitorar'], recommendations['Controle 9M'], recommendations['Controle 3M'], recommendations['Outra desfolha']]
})

# TABELA RESUMO MONITORAMENTO

grouped_farm_date_tabela = grouped_farm_date[['FARM', 'farm_total_area_ha']].reset_index(drop=True)
grouped_farm_date_tabela = grouped_farm_date_tabela.rename(columns={
    'FARM': 'Fazenda',
    'farm_total_area_ha': 'Área monitorada (ha)'
})

# TABELA RESUMO MONITORAMENTO FAZENDA

grouped_stand_date_tabela = grouped_stand_date[['STAND', 'stand_total_area_ha']].reset_index(drop=True)
grouped_stand_date_tabela = grouped_stand_date_tabela.rename(columns={
    'STAND': 'Talhão',
    'stand_total_area_ha': 'Área monitorada (ha)'
})

grouped_stand_date_tabela

,Talhão,Área monitorada (ha)
0,BOI_PRETO_XI_055,75.6
1,BOI_PRETO_XI_031,77.2
2,BOI_PRETO_XI_056,54.1
3,EMBAY_003,67.0
4,EMBAY_027,55.3
...,...,...
172,EMBAY_038,28.2
173,EMBAY_083,26.4
174,EMBAY_034,18.4
175,EMBAY_050,47.7


In [32]:

# BORDA ARREDONDADA

def bg_border(color):
    st.markdown(
        f"""
        <style>
        .stPlotlyChart {{
        outline: 3px solid {color};
        border-radius: 5px;
        box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.20), 0 6px 20px 0 rgba(0, 0, 0, 0.30);
        }}
        </style>
        """, unsafe_allow_html=True
    )

# CARDS ESTILO

def create_card(title, value):
    return f"""
    <div style="
        background-color: #f5f5f5;
        border-radius: 10px;
        padding: 20px;
        margin: 10px;
        box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.1);
        text-align: center;
        font-family: Arial, sans-serif;
    ">
        <h4 style="margin: 0; color: #333; font-size: 20px;">{title}</h4>
        <h1 style="margin: 0; color: #000000;font-size: 35px;">{value}</h1>
    </div>
    """

# CARD ÁREA TOTAL MONITORADA

total_area_df = stands_all[stands_all['COMPANY'] == empresa]
total_area_m2 = total_area_df['geometry'].to_crs("EPSG:32722").area.sum()
total_area_ha = total_area_m2 / 10000
total_area_ha_rounded = round(total_area_ha, 1)

# CARD ÁREA TOTAL DESFOLHA

total_area_desfolha = grouped_farm_date['farm_desfolha_area_ha'].sum()
total_area_desfolha_rounded = round(total_area_desfolha, 1)

# CARD NÚMERO DE FAZENDAS NA EMPRESA

filtered_farms = stands_all[stands_all['COMPANY'] == empresa]
num_unique_farms = filtered_farms['FARM'].nunique()

# CARD NÚMERO DE TALHOES NA EMPRESA

num_unique_stands = filtered_farms['STAND'].nunique()

# CARD ÁREA FAZENDA 

farm_area_df = stands_all[stands_all['FARM'] == fazenda]
farm_area_m2 = farm_area_df['geometry'].to_crs("EPSG:32722").area.sum()
farm_area_ha = farm_area_m2 / 10000
farm_area_ha_rounded = round(farm_area_ha, 1)

# CARD ÁREA DESFOLHA FAZENDA

farm_area_desfolha = grouped_farm_date[grouped_farm_date['FARM']==fazenda]['farm_desfolha_area_ha'].sum()
farm_area_desfolha_rounded = round(farm_area_desfolha, 1)

# CARD NÚMERO DE TALHÕES NA FAZENDA

filtered_stands = stands_all[stands_all['FARM'] == fazenda]
num_unique_stands_farm = filtered_stands['STAND'].nunique()

# CARD ÁREA TOTAL TALHÃO

stand_area_df = stands_all[stands_all['STAND'] == talhao]
stand_area_m2 = stand_area_df['geometry'].to_crs("EPSG:32722").area.sum()
stand_area_ha = stand_area_m2 / 10000
stand_area_ha_rounded = round(stand_area_ha, 1)

# CARD ÁREA DESFOLHA TALHÃO

stand_area_desfolha = grouped_stand_date[grouped_stand_date['STAND']==talhao]['stand_desfolha_area_ha'].sum()
stand_area_desfolha_rounded = round(stand_area_desfolha, 1)

# GRÁFICO DE ROSCA ÁREA MONITORADA  

# Definindo variáveis auxiliares
farm_percentage = (farm_area_m2 / total_area_m2) * 100
healthy_percentage = 100 - farm_percentage
other_area_ha = total_area_ha - farm_area_ha

# Definindo parâmetros para o gráfico
sizes = [other_area_ha, farm_area_ha]
colors = ['lightgray', 'darkgreen']
labels = ['Demais fazendas', fazenda]

# Calculate total desfolha and healthy areas
total_area_desfolha = grouped_farm_date['farm_desfolha_area_ha'].sum()
total_area_healthy = total_area_ha - total_area_desfolha

# Define labels and sizes for the pie chart
labels = ['Área Saudável', 'Área em Desfolha']
sizes = [total_area_healthy, total_area_desfolha]
colors = ['darkgreen', 'red']  # Adjust colors to your preference

# Generate the pie chart with custom labels and colors
fig1 = go.Figure()

fig1.add_trace(go.Pie(
    labels=['Área Total Saudável', 'Área Total c/ Desfolha'],
    values=[total_area_healthy, total_area_desfolha],
    marker=dict(colors=['darkgreen', 'red']),
    textinfo='percent',
    textfont=dict(size=20),  # Increase percentage text size inside the pie chart
    hovertemplate=(
        '<b>%{label}</b><br>'
        'Área: %{value:.2f} ha<br>'
        'Porcentagem: %{percent:.1%}<extra></extra>'
    ),
    showlegend=True
))

# Update the layout to move the legend to the bottom of the page
fig1.update_layout(
    title={
        'text': "Área Monitorada",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'color': 'black', 'size': 21}
    },
    paper_bgcolor='#f5f5f5',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=True,
    legend=dict(
        orientation="h",  # Horizontal legend
        yanchor="top",
        y=-0.3,  # Move legend farther below the pie chart (adjust if needed)
        xanchor="center",
        x=0.5,
        font=dict(size=14, color='black'),
        bgcolor='rgba(0,0,0,0)'  # Transparent background for the legend
    )
)

# Calculate desfolha percentage and healthy area
grouped_farm_date['percentage_desfolha'] = (
    grouped_farm_date['farm_desfolha_area_ha'] / grouped_farm_date['farm_total_area_ha']
) * 100

grouped_farm_date['healthy_area_ha'] = (
    grouped_farm_date['farm_total_area_ha'] - grouped_farm_date['farm_desfolha_area_ha']
)

# Sort by percentage of desfolha
grouped_farm_date = grouped_farm_date.sort_values(by='percentage_desfolha', ascending=False)

# Transform data to long format
grouped_farm_date_long = grouped_farm_date.melt(
    id_vars=['FARM', 'percentage_desfolha'],
    value_vars=['farm_desfolha_area_ha', 'healthy_area_ha'],
    var_name='Tipo de Área',
    value_name='Área (ha)'
)

# Update labels for the chart
grouped_farm_date_long['Tipo de Área'] = grouped_farm_date_long['Tipo de Área'].map({
    'farm_desfolha_area_ha': 'Área de Desfolha (ha)',
    'healthy_area_ha': 'Área Saudável (ha)'
})

grouped_farm_date_long

,FARM,percentage_desfolha,Tipo de Área,Área (ha)
0,EMBAY,1536.962025,Área de Desfolha (ha),849.94
1,BOI_PRETO_XI,762.791005,Área de Desfolha (ha),576.67
2,EMBAY,1536.962025,Área Saudável (ha),-794.64
3,BOI_PRETO_XI,762.791005,Área Saudável (ha),-501.07


In [34]:
# TALHÕES MAIS INFESTADOS GERAL

# Calculate percentage of desfolha for each stand
grouped_stand_date['desfolha_percentage'] = (grouped_stand_date['stand_desfolha_area_ha'] / grouped_stand_date['stand_total_area_ha']) * 100

# Sort by percentage and get the top 10
top_10_stands_geral = grouped_stand_date.sort_values(by='desfolha_percentage', ascending=False).head(10)

# Define gradient colors for the bars
colors = ["#FF0000", "#FF2200", "#FF4400", "#FF6600", "#FF8800", "#FFAA00", "#FFBB00", "#FFCC00", "#FFDD33", "#FFEE66"]

# Generate the bar chart
fig3 = go.Figure()
fig3.add_trace(go.Bar(
    x=top_10_stands_geral['desfolha_percentage'],
    y=top_10_stands_geral['STAND'],
    orientation='h',
    marker=dict(color=colors),
    text=top_10_stands_geral['desfolha_percentage'].round(1).astype(str) + '%',  # Display percentage text
    textposition='auto'
))

# Adjust chart layout
fig3.update_layout(
    title='Top 10 Talhões com Maior Percentual de Desfolha',
    xaxis_title=dict(text="Percentual de Desfolha (%)", font=dict(size=14, color='black')),
    yaxis_title=dict(text="Talhão", font=dict(size=14, color='black')),
    title_font=dict(size=16, family='Arial', color='black'),
    xaxis=dict(tickfont=dict(size=12, color='black')),
    yaxis=dict(tickfont=dict(size=12, color='black'), autorange='reversed'),
    paper_bgcolor='#f5f5f5',
    plot_bgcolor='rgba(0,0,0,0)'
)

# Variável auxiliar
healthy_area_farm = farm_area_ha_rounded - farm_area_desfolha_rounded

# Definindo parâmetros para o gráfico
sizes = [farm_area_desfolha_rounded, healthy_area_farm]
colors = ['red', 'darkgreen']
labels = ['Área Total c/ Desfolha', 'Área Total Saudável']

# Gerando o gráfico (Pie Chart)
fig4 = go.Figure()
fig4.add_trace(go.Pie(
    labels=labels,
    values=sizes,
    marker=dict(colors=colors),
    textinfo='percent',
    texttemplate='%{percent:.1%}',  # Percentages displayed inside
    insidetextorientation='horizontal',  # Horizontal text orientation
    hovertemplate=(
        '<b>%{label}</b><br>'
        'Área: %{value:.2f} ha<br>'
        'Porcentagem: %{percent:.1%}<extra></extra>'
    ),
    textfont=dict(size=20, color='white')  # Larger white text for percentages
))

# Ajustando o visual do gráfico
fig4.update_layout(
    title={'text': "Área Monitorada", 'y': 0.95, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top', 'font': {'color': 'black', 'size': 21, 'family': 'Arial Black'}},
    paper_bgcolor='#f5f5f5',
    plot_bgcolor='rgba(0,0,0,0)',
    showlegend=True,
    legend=dict(
        orientation='h',          # Horizontal legend at the bottom
        yanchor='bottom',
        y=-0.2,                   # Position below the chart
        xanchor='center',
        x=0.5,
        font=dict(size=14, color='black'),
        bgcolor='rgba(0, 0, 0, 0)'
    )
)

# Base auxiliar: Calculate percentage of desfolha for each stand
grouped_stand_farm['desfolha_percentage'] = (grouped_stand_farm['stand_desfolha_area_ha'] / grouped_stand_farm['stand_total_area_ha']) * 100

# Sort by percentage in descending order and get the top 10 stands
top_10_stands_farm = grouped_stand_farm.sort_values(by='desfolha_percentage', ascending=False).head(10)

# Define colors for the gradient (optional)
colors = ["#FF0000", "#FF2200", "#FF4400", "#FF6600", "#FF8800", "#FFAA00", "#FFBB00", "#FFCC00", "#FFDD33", "#FFEE66"]

top_10_stands_geral


,DATE,Status,FARM,STAND,count,stand_total_area_ha,stand_desfolha_area_ha,total,percentage,desfolha_percentage
944,2024-10-01,Desfolha,EMBAY,EMBAY_099G,2781,18.0,27.81,2784,99.9,154.500000
961,2024-10-01,Desfolha,EMBAY,EMBAY_115R,2268,15.5,22.68,2322,97.7,146.322581
948,2024-10-01,Desfolha,EMBAY,EMBAY_102A,1222,9.0,12.22,1238,98.7,135.777778
856,2024-10-01,Desfolha,EMBAY,EMBAY_009,3685,46.9,36.85,3797,97.1,78.571429
845,2024-10-01,Desfolha,BOI_PRETO_XI,BOI_PRETO_XI_056,4250,54.1,42.50,4412,96.3,78.558226
960,2024-10-01,Desfolha,EMBAY,EMBAY_115,1657,21.5,16.57,1657,100.0,77.069767
955,2024-10-01,Desfolha,EMBAY,EMBAY_109,2314,30.2,23.14,2321,99.7,76.622517
874,2024-10-01,Desfolha,EMBAY,EMBAY_027,4148,55.3,41.48,4170,99.5,75.009042
958,2024-10-01,Desfolha,EMBAY,EMBAY_112,1894,25.3,18.94,1966,96.3,74.861660
962,2024-10-01,Desfolha,EMBAY,EMBAY_116,2263,30.4,22.63,2263,100.0,74.440789
